# Twitter Web Scrapper 

## Importing Modules

In [15]:
#Selenium Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchWindowException
#Threading library
import threading
#Pandas
import pandas as pd
#Datetime Modules
from datetime import datetime
import time

## Twitter Scrapper

In [2]:
def twitterScrapper(link, credentials, searchFilter, dataSet):    
    driverOpts = Options()

    driverOpts.headless = True
    
    driver = webdriver.Chrome(options=driverOpts)

    driver.get(link)
    try:
        # Login to twitter
        waitToLoad = WebDriverWait(driver, 20)
        waitToLoad.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[autocomplete="username"]'))).send_keys(credentials[0])
        driver.find_element(By.XPATH, ("//*[contains(text(), 'Next')]")).click();
        waitToLoad.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[autocomplete="current-password"]'))).send_keys(credentials[1])
        waitToLoad.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[data-testid='LoginForm_Login_Button'][role='button']"))).click()
        
        #Add Search Filter
        searchBar = waitToLoad.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[data-testid='SearchBox_Search_Input'][role='combobox']")))
        searchBar.send_keys(searchFilter)
        searchBar.send_keys(Keys.ENTER)
        
        # Sort by Latest
        latestTab = waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[2]//*[contains(text(), 'Latest')]")))
        latestTab.click()
        
        #Scrapping Tweets
        scrappedUserNames = []
        scrappedTweets = []
        scrappedTweetDate = []

        prevHeight = driver.execute_script('return document.body.scrollHeight')

        while True:
            time.sleep(10)

            users = driver.find_elements(By.XPATH, ("//div[@data-testid='User-Name']/div[1]/div[1]/a"))
            tweets = driver.find_elements(By.XPATH, ("//div[@data-testid='tweetText']"))
            tweetDates = driver.find_elements(By.XPATH, ("//div[@data-testid='User-Name']/div[2]/div[1]/div[3]/a/time"))
            
            scrappedUserNames = [user.text for user in users]
                
            scrappedTweets = [tweet.text for tweet in tweets]
                
            scrappedTweetDate = [date.text for date in tweetDates]

            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            currentHeight = driver.execute_script('return document.body.scrollHeight')

            if (currentHeight == prevHeight) or (len(scrappedTweets) > 200):
                break
            
            prevHeight = currentHeight
            
    except NoSuchWindowException:
          print("Selenium window closed unexpectedly.")
        


    
    for i in range(len(scrappedUserNames)):
        dataSet[scrappedUserNames[i]] = [scrappedTweets[i], scrappedTweetDate[i]]

In [23]:
linkTwitter = "https://twitter.com/i/flow/login"
twitterCredentials = ["ScrpperR16322", "Test8dcln"]
searchFilter = '("business confidence" OR "economic growth" OR "fiscal policy" OR "monetary policy" OR "interest rates" OR "income inequality" OR "financial stability" OR "labor market" OR "economic indicators" OR "economic recovery" OR "cost of living") until:2009-01-01 since:2007-01-01 -filter:links -filter:replies'
searchFilter2 = '("economy" OR "economic" OR "job market" OR "unemployment" OR "inflation" OR "recession" OR "stock market" OR "GDP" OR "consumer spending" ) until:2009-01-01 since:2007-01-01 -filter:links -filter:replies'
scrappedTwitterData = {}

th1 = threading.Thread(target=twitterScrapper, args=(linkTwitter, twitterCredentials, searchFilter, scrappedTwitterData))
th2 = threading.Thread(target=twitterScrapper, args=(linkTwitter, twitterCredentials, searchFilter2, scrappedTwitterData))


th1.start()
th2.start()

th1.join()
th2.join()

Selenium window closed unexpectedly.
Selenium window closed unexpectedly.


## Facebook Scrapper

In [39]:
def isValidDate(text):
    try:
        datetime.strptime(text, '%d %B %Y')
        return True
    except ValueError:
        return False

def facebookScrapper(link, searchKeywords, credentials, dataSet, searchDates):
     driverOpts = Options()

     driverOpts.headless = False

     driver = webdriver.Chrome(options=driverOpts)

     driver.get(link)
     try:
          # Login to facebook
          waitToLoad = WebDriverWait(driver, 20)
          waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "//input[@data-testid='royal_email']"))).send_keys(credentials[0])
          waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "//input[@data-testid='royal_pass']"))).send_keys(credentials[1])
          waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='royal_login_button']"))).click()
          
          scrappedFacebookUserNames = []
          scrappedFacebookPosts = []
          scrappedFacebookDate = []
          
          waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "/html"))).click()
          
          # Scrapping
          for keyword in searchKeywords:
               for date in searchDates:
                    time.sleep(5)
                    #Searching
                    searchFacebook = waitToLoad.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[placeholder="Search Facebook"]')))
                    searchFacebook.clear()
                    searchFacebook.send_keys(keyword)
                    searchFacebook.send_keys(Keys.ENTER)
                    
                    #Selecting Search Criteria
                    waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Posts')]"))).click()
                    waitToLoad.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Date posted')]"))).click()
                    waitToLoad.until(EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(),'{date}')]"))).click()
                    
                    #Selecting and Storing information
                    prevHeight = driver.execute_script('return document.body.scrollHeight')
                    while True:
                         time.sleep(5)

                         fbUsers = driver.find_elements(By.XPATH, "//a[contains(@class, 'x1i10hfl') and contains(@class, 'xjbqb8w') and contains(@class, 'x1ejq31n') and contains(@class, 'xd10rxx') and contains(@class, 'x1sy0etr') and contains(@class, 'x17r0tee') and contains(@class, 'x972fbf') and contains(@class, 'xcfux6l') and contains(@class, 'x1qhh985') and contains(@class, 'xm0m39n') and contains(@class, 'x9f619') and contains(@class, 'x1ypdohk') and contains(@class, 'xt0psk2') and contains(@class, 'xe8uvvx') and contains(@class, 'xdj266r') and contains(@class, 'x11i5rnm') and contains(@class, 'xat24cr') and contains(@class, 'x1mh8g0r') and contains(@class, 'xexx8yu') and contains(@class, 'x4uap5') and contains(@class, 'x18d9i69') and contains(@class, 'xkhd6sd') and contains(@class, 'x16tdsg8') and contains(@class, 'x1hl2dhg') and contains(@class, 'xggy1nq') and contains(@class, 'x1a2a7pz') and contains(@class, 'xt0b8zv') and contains(@class, 'xzsf02u') and contains(@class, 'x1s688f')]")
                         fbDates = driver.find_elements(By.XPATH, "//a[contains(@class, 'x1i10hfl') and contains(@class, 'xjbqb8w') and contains(@class, 'x1ejq31n') and contains(@class, 'xd10rxx') and contains(@class, 'x1sy0etr') and contains(@class, 'x17r0tee') and contains(@class, 'x972fbf') and contains(@class, 'xcfux6l') and contains(@class, 'x1qhh985') and contains(@class, 'xm0m39n') and contains(@class, 'x9f619') and contains(@class, 'x1ypdohk') and contains(@class, 'xt0psk2') and contains(@class, 'xe8uvvx') and contains(@class, 'xdj266r') and contains(@class, 'x11i5rnm') and contains(@class, 'xat24cr') and contains(@class, 'x1mh8g0r') and contains(@class, 'xexx8yu') and contains(@class, 'x4uap5') and contains(@class, 'x18d9i69') and contains(@class, 'xkhd6sd') and contains(@class, 'x16tdsg8') and contains(@class, 'x1hl2dhg') and contains(@class, 'xggy1nq') and contains(@class, 'x1a2a7pz') and contains(@class, 'x1heor9g') and contains(@class, 'xt0b8zv') and contains(@class, 'xo1l8bm')]")
                         fbPosts = fbPosts = driver.find_elements(By.XPATH, "//div[contains(@class, 'xdj266r') and contains(@class, 'x11i5rnm') and contains(@class, 'xat24cr') and contains(@class, 'x1mh8g0r') and contains(@class, 'x1vvkbs')]")
                                        
                         scrappedFacebookDate = [item.text for item in fbDates if isValidDate(item.text)]
                              
                         scrappedFacebookUserNames = [user.text for user in fbUsers if user.text != "See more"]
                              
                         scrappedFacebookPosts = [post.text for post in fbPosts]

                         driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                         currentHeight = driver.execute_script('return document.body.scrollHeight')

                         if (currentHeight == prevHeight) or (len(scrappedFacebookPosts) > 200):
                              break
                         
                         prevHeight = currentHeight
                         
                         print(scrappedFacebookUserNames)
                         print(scrappedFacebookPosts)
                         print(scrappedFacebookDate)
               
     except NoSuchWindowException:
          print("Selenium window closed unexpectedly.")
          
     print(len(scrappedFacebookPosts))
     print(len(scrappedFacebookUserNames))

In [42]:
linkFacebook = "https://www.facebook.com/"
searchKeywords = ['economy', 'economic', 'job market', 'unemployment', 'inflation', 'recession', 'stock market', 'GDP', 'consumer spending', 'business confidence', 'economic growth', 'fiscal policy', 'monetary policy', 'interest rates', 'income inequality', 'financial stability', 'labor market', 'economic indicators', 'economic recovery', 'cost of living', "job loss", "homeless"]
facebookCredentials = ["fawdaclo@gmail.com", "Test8dcln"]
searchDates = [2007,2008]
scrappedFacebookData = {}

facebookScrapper(linkFacebook, searchKeywords, facebookCredentials, scrappedFacebookData, searchDates)

['The Onion', 'Fox News', 'Kamala Harris', 'http://www.facebook.com/video/video.php?v=553058559053', 'http://www.facebook.com/video/video.php?v=553063234683', 'Fox News']
["With the economy sliding deeper into a recession, panelists discuss whether it's time to stop throwing our money into a massive pit out in the desert.", 'John McCain and Barack Obama go head-to-head on  the economy and foreign policy at the first presidential debate', 'One Economy\'s "If I Were President"', 'President Bush on state of the economy']
['13 November 2008', '27 September 2008', '31 August 2008', '25 September 2008']
['The Onion', 'Fox News', 'Kamala Harris', 'http://www.facebook.com/video/video.php?v=553058559053', 'http://www.facebook.com/video/video.php?v=553063234683', 'Fox News', 'Daily Naya Diganta']
["With the economy sliding deeper into a recession, panelists discuss whether it's time to stop throwing our money into a massive pit out in the desert.", 'John McCain and Barack Obama go head-to-head o

KeyboardInterrupt: 

## Creating CSV File From Data

In [24]:
df = pd.DataFrame.from_dict(scrappedTwitterData, orient='index')

df.reset_index(inplace=True)
# df.columns = ['username', 'tweet_date']

# df[['post', 'date']] = pd.DataFrame(df['tweet_date'].tolist(), index=df.index)

# df.drop(columns=['tweet_date'], inplace=True)

df['platform'] = 'X/Twitter'

csv_file = 'data.csv'
df.to_csv(csv_file, index=False)

print("CSV file created successfully!")
print(len(scrappedTwitterData))


CSV file created successfully!
115
